In [ ]:
import pandas as pd
import statistics
import altair as alt

In [ ]:
scores = '../Data/20250825_BARD1snvscores_filtered.xlsx' #path to SGE scores
func_cutoff = 0.8 #functional class GMM cutoff (min)
nf_cutoff = 0.65 #non-functional class GMM cutoff (max)
BARD1_WT = 'MPDNRQPRNRQPRIRSGNEPRSAPAMEPDGRGAWAHSRAALDRLEKLLRCSRCTNILREPVCLGGCEHIFCSNCVSDCIGTGCPVCYTPAWIQDLKINRQLDSMIQLCSKLRNLLHDNELSDLKEDKPRKSLFNDAGNKKNSIKMWFSPRSKKVRYVVSKASVQTQPAIKKDASAQQDSYEFVSPSPPADVSERAKKASARSGKKQKKKTLAEINQKWNLEAEKEDGEFDSKEESKQKLVSFCSQPSVISSPQINGEIDLLASGSLTESECFGSLTEVSLPLAEQIESPDTKSRNEVVTPEKVCKNYLTSKKSLPLENNGKRGHHNRLSSPISKRCRTSILSTSGDFVKQTVPSENIPLPECSSPPSCKRKVGGTSGRKNSNMSDEFISLSPGTPPSTLSSSSYRRVMSSPSAMKLLPNMAVKRNHRGETLLHIASIKGDIPSVEYLLQNGSDPNVKDHAGWTPLHEACNHGHLKVVELLLQHKALVNTTGYQNDSPLHDAAKNGHVDIVKLLLSYGASRNAVNIFGLRPVDYTDDESMKSLLLLPEKNESSSASHCSVMNTGQRRDGPLVLIGSGLSSEQQKMLSELAVILKAKKYTEFDSTVTHVVVPGDAVQSTLKCMLGILNGCWILKFEWVKACLRRKVCEQEEKYEIPEGPRRSRLNREQLLPKLFDGCYFYLWGTFKHHPKDNLIKLVTAGGGQILSRKPKPDSDVTQTINTVAYHARPDSDQRFCTQYIIYEDLCNYHPERVRQGKVWKAPSSWFIDCVMSFELLPLDS'
alt.data_transformers.disable_max_rows()

In [ ]:
def read_scores(file): #Reads SGE scores
    df = pd.read_excel(file)
    df = df.rename(columns = {'consequence': 'Consequence', 'score': 'snv_score', 'amino_acid_change': 'AAsub'})
    
    df = df.loc[df['Consequence'].isin(['missense_variant', 'stop_gained', 'synonymous_variant'])]

    return df

In [ ]:
def process_scores(df, func, nf): #Processes scores. Gets original AA, substituted AA, and functional classification
    
    df['og_AA'] = df['AAsub'].transform(lambda x: x[0]) #Makes column with the original amino acid
    df['AA_change'] = df['AAsub'].transform(lambda x: x[-1]) #makes column with amino acid change
    df['AApos'] = df['AAsub'].transform(lambda x: x[1: len(x)-1]) #makes column with residue position

    aa_grouped =df.groupby('AAsub') #groups by amino acid substitution
 
    aa_pos = [] #empty list to hold amino acid positions
    full_sub = [] #empty list for substitution
    og_aa = [] #empty list to hold original amino acids
    sub_aa = [] #empty list to hold substituted amino acids
    sub_score = [] #empty list to hold scores for each changge
    function = [] #empty list to hold functional classification
    
    for aa_sub, vars in aa_grouped:
        full_sub.append(aa_sub)
        vars = vars.reset_index(drop = True) #resets index for each grouopby object
        aa_pos.append(int(vars['AApos'][0])) #appends amino position
        og_aa.append(vars['og_AA'][0]) #appends original amino acid
        sub_aa.append(vars['AA_change'][0]) #appends substituted amino acid

        score_list = vars['snv_score'].tolist() #makes list of SGE scores for that amino acid position
        median_score = statistics.median(score_list)
        sub_score.append(median_score)

        if median_score > func:
            function.append('Functional')
        elif nf < median_score < func:
            function.append('Intermediate')
        else:
            function.append('Non-Functional')


    df = pd.DataFrame({'AApos': aa_pos, 'og_AA': og_aa,
                        'AAsub': sub_aa, 'score': sub_score,
                       'full_sub': full_sub, 'function': function
                      }) #output dataframe

    return df

In [ ]:
def get_min_res_score(df): #Gets minimum missense scores for each residue position

    df = df.loc[df['Consequence'].isin(['missense_variant'])]
    aa_grouped_df = df.groupby('AApos')['snv_score'].min().reset_index() #gets the mean missense SGE score for each amino acid position

    aa_grouped_df = aa_grouped_df.rename(columns = {'snv_score': 'min_snv_score'})
    
    #makes bar chart that displays min score vs. amino acid position
    sge_min_bar = alt.Chart(aa_grouped_df).mark_bar().encode(
        x = alt.X('AApos:Q', axis = alt.Axis(title = None, 
                                             labels = False, 
                                             ticks = False,
                                             domain = True
                                            ),
                  scale = alt.Scale(domain = [0,777])), 
        y = alt.Y('min_snv_score:Q', axis = alt.Axis(title = 'Min. Missense Score', labelFontSize = 12, titleFontSize = 14)),
        tooltip = [alt.Tooltip('AApos', title = 'Position: '),
                   alt.Tooltip('min_snv_score', title = 'Min. Missense Score: ')]
    ).properties(
        width = 1500,
        height = 150,
        title = alt.TitleParams(text = '', fontSize = 22)
    )

    #sge_min_bar.display()

    return sge_min_bar, aa_grouped_df

In [ ]:
def get_mean_res_score(df): #Gets mean missense score for each residue position

    df = df.loc[df['Consequence'].isin(['missense_variant'])]
    aa_grouped_df = df.groupby('AApos')['snv_score'].mean().reset_index() #gets the mean missense SGE score for each amino acid position

    aa_grouped_df = aa_grouped_df.rename(columns = {'snv_score': 'mean_snv_score'})
    
    #makes bar chart that displays mean score vs. amino acid position
    sge_mean_bar = alt.Chart(aa_grouped_df).mark_bar().encode(
        x = alt.X('AApos:Q', scale = alt.Scale(domain = [0,777]),
                  axis = alt.Axis(title = 'Amino Acid Position', 
                                  labelFontSize = 12, 
                                  titleFontSize = 14
                                 )
                 ),
        y = alt.Y('mean_snv_score:Q', axis = alt.Axis(title = 'Mean Missense Score', labelFontSize = 12, titleFontSize = 14)),
        tooltip = [alt.Tooltip('AApos', title = 'Position: '),
                   alt.Tooltip('mean_snv_score', title = 'Mean Missense Score: ')]
    ).properties(
        width = 1500,
        height = 150,
        title = alt.TitleParams(text = '', fontSize = 22)
    )

    #sge_mean_bar.display()

    return sge_mean_bar, aa_grouped_df

In [ ]:
def min_mean_stacked(min_df, mean_df): #Builds min-mean stacked bar chart
    
    df = pd.merge(min_df, mean_df, how = 'inner')

    df_long = df.melt(
        id_vars=['AApos'],
        value_vars=['min_snv_score', 'mean_snv_score'],
        var_name='Series',
        value_name='Value'
    )

    chart = alt.Chart(df_long).mark_bar(opacity = 0.7).encode(
        x=alt.X('AApos:Q', scale = alt.Scale(domain = [0, 777]), title= 'Position'),
        y=alt.Y('Value:Q', stack=True),
        color=alt.Color('Series:N', legend=alt.Legend(title='Values')),
        tooltip=['AApos', 'Series', 'Value']
    ).properties(
        width=1500,
        height=200
    )
    
    return chart

In [ ]:
def append_mean_min(sub_df, min_df, mean_df): #Appends residue-level min and mean scores to main dataframe

    #Create shared columns
    min_df['AAsub'] = 'Mis. Min.'
    mean_df['AAsub'] = 'Mis. Mean'

    min_df['og_AA'] = 'Mis. Min.'
    mean_df['og_AA'] = 'Mis. Mean'

    min_df['full_sub'] = 'Mis. Min.'
    mean_df['full_sub'] = 'Mis. Mean'

    min_df['function'] = 'Mis. Min.'
    mean_df['function'] = 'Mis. Mean'

    min_df.AApos = min_df.AApos.astype(int) #set datatype as int
    mean_df.AApos = mean_df.AApos.astype(int) #set datatype as int
    
    min_df = min_df.rename(columns = {'min_snv_score': 'score'}) #Renames to yield shared columns
    mean_df = mean_df.rename(columns = {'mean_snv_score': 'score'}) #renames to yield shared columns

    df = pd.concat([sub_df,min_df, mean_df]) #concatenates dataframes
    df.loc[(df['AAsub'] == '*', 'AAsub')] = 'Stop' #Renames stop-gained variants
    return df

In [ ]:
def add_wt_seq(df, wt_seq): #Adds the WT seqeuence to dataframe
    wt = [] #list to hold WT sequence
    
    for char in wt_seq:
        wt.append(char) #Splits up large string into separate characters

    #Creates DataFrame with WT sequence
    wt_data = pd.DataFrame({
        'AApos': range(1, len(wt) + 1),
        'og_AA': wt,
        'AAsub': wt, 
        'score': 1,
        'full_sub': 'WT',
        'is_wt': True
    })

    df['is_wt'] = False #Sets WT to False in SGE dataframe

    df_w_WT = pd.concat([df, wt_data], ignore_index = True) #concatenates dataframes

    return df_w_WT

In [ ]:
def heatmap(df): #Builds heatmap without WT sequence


    order = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'Stop', 'Min.', 'Mean']
        
    map = alt.Chart(df).mark_rect().encode(
        x = alt.X('AApos:N', axis=alt.Axis(
                values=[i for i in range(0, 777, 100)],
                title=None,  # removes title
                labels=False,  # removes labels
                ticks=False,  # removes ticks
                domain=True  # keeps the domain line,
        )),
        y = alt.Y('AAsub:N', sort = order,
                  axis = alt.Axis(title = 'Amino Acid Substitution', titleFontSize = 16)),
        color = alt.condition(
            alt.datum.score <= -0.5,  # Replace X with your lower threshold
            alt.value('#ff0000'),  # Pure red for values <= X
            alt.condition(
                alt.datum.score >= 0,  # Replace Y with your upper threshold
                alt.value('#0000ff'),  # Pure blue for values >= Y
                alt.Color('score:Q',  # Gradient for values between X and Y
                         scale=alt.Scale(
                             domain=[-0.5, 0],
                             range=['#ff0000', '#a6a6a6', '#0000ff']
                         ))
            )
        ),
        tooltip = [alt.Tooltip('AApos', title = 'Position: '),
                   alt.Tooltip('full_sub', title = 'Substitution: '),
                   alt.Tooltip('score', title = 'SGE Score: ')]
    ).properties(
        height = 700, 
        width = 1500
    ).interactive()

    
    #Things for rectanges for domains - probably easier to just draw them in
    rects_df = pd.DataFrame({
        'start': [26, 426, 566],
        'end': [120, 560, 777],
        'y': [10,10,10]
    })
    domain_rects = alt.Chart(rects_df).mark_rect(
        height = 20
    ).encode(
        x = 'start:Q',
        x2 = 'end:Q',
        color = alt.value('gray')
    )
    
    map.display()

    return map
    #domain_rects.display()

    #domain_heatmap = alt.layer(domain_rects, map)

    #domain_heatmap.display()

In [ ]:
def heatmap_WT(df): #Builds heatmap with WT sequence in black

    
    df = df.loc[~(df['full_sub'].isin(['*778*']))]

    order = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'Stop', 'Min.', 'Mean'] #order for heatmap y-axis


    annotation_data = pd.DataFrame([
        {'start': 26, 'end': 122, 'label': 'RING', 'color': '#B9DBF4'},
        {'start': 425, 'end': 545, 'label': 'ARD', 'color': '#C8DBC8'},
        {'start': 568, 'end': 777, 'label': 'BRCT', 'color': '#F6BF93'},
        # Add more annotations as needed
    ])

    rect_colors = ['#B9DBF4','#C8DBC8','#F6BF93']
    domains = ['RING', 'ARD', 'BRCT']

    # Calculate center positions for text
    annotation_data['center'] = (annotation_data['start'] + annotation_data['end']) / 2

    
    # Create domain rectangles
    annotation_rect = alt.Chart(annotation_data).mark_rect(height=30, 
                                                           stroke = 'black',
                                                          strokeWidth = 2 ).encode(
        x=alt.X('start:Q',
                axis = None,
                scale=alt.Scale(domain=[0, 777])),
        x2='end:Q',
        color=alt.Color('label:N', 
                        scale = alt.Scale(domain = domains,
                                          range = rect_colors
                                         ),
                        legend= None),
        tooltip=['label', 'start', 'end']
    ).properties(
        width=1750,
        height=20
    )

    #Domain text labels
    annotation_text = alt.Chart(annotation_data).mark_text(
        color='black',
        fontSize=24,
        fontWeight='bold',
        baseline='middle',
        dy = -10 # This helps with vertical centering
    ).encode(
        x=alt.X('center:Q', 
                scale=alt.Scale(domain=[0, 777]),
                axis=None
        ), # Position text in the middle of the 50px height
        text='label:N'
    )

    annotations = alt.layer(annotation_rect, annotation_text).properties(
        width=1750,
        height=20
    )


    #for heatmap
    map = alt.Chart(df).mark_rect().encode(
    x = alt.X('AApos:N', axis=alt.Axis(
            values=[i for i in range(0, 777, 100)],
            title='Amino Acid Residue', titleFontSize = 32, labelFontSize = 28,
            labels=True,
            ticks= True,
            domain=True
    )),
    y = alt.Y('AAsub:N', sort = order,
              axis = alt.Axis(title = 'Amino Acid Substitution', titleFontSize = 32, labelFontSize = 28)),
    color = alt.condition(
        'datum.is_wt == true',
        alt.value('#000000'),  # Wild type in black
        alt.Color('score:Q', title = 'SGE Score',
                  scale = alt.Scale(
                      domain = [-0.3, 0, 0.15],  # Add upper bound for positive values
                      range = ['#ff0000', '#a6a6a6', '#a6a6a6']  # Keep gray for 0 and positive values
                  ),
                  legend = alt.Legend(titleFontSize = 24, 
                                      labelFontSize = 22)
        )
    ),
    tooltip =[alt.Tooltip('AApos', title = 'Position: '),
               alt.Tooltip('full_sub', title = 'Substitution: '),
               alt.Tooltip('score', title = 'SGE Score: ')]
    ).properties(
    height = 800, 
    width = 1750
    ).interactive()

    #for WT legend
    legend_data = pd.DataFrame([{'label': 'WT'}])
    legend = alt.Chart(legend_data).mark_rect().encode(
        y=alt.Y('label:N', title = ''),
        color=alt.value('#000000')
    ).properties(
        title=''
    )
    
    # Combine the charts
    heatmap_with_legend = alt.hconcat(map, legend).resolve_scale(color='independent')
    final_chart = alt.vconcat(annotations, heatmap_with_legend, spacing= -5)

    #final_chart.save('/Users/ivan/Desktop/BARD1_draft_figs/fig_2b_heatmap.png', ppi = 500)
    #legend.save('/Users/ivan/Desktop/BARD1_draft_figs/fig_2b_WTlegend.png', ppi = 500)
    
    final_chart = final_chart.configure_axis(
        grid = False
    ).configure_view(
        stroke = None
    ).resolve_axis(
        x = 'shared'
    )

    legend = legend.configure_view(
        stroke = None
    ).configure_axis(
        grid = False
    )
    
    #final_chart.save('/Users/ivan/Desktop/BARD1_draft_figs/fig_2b_heatmap.png', ppi = 500)
    #legend.save('/Users/ivan/Desktop/BARD1_draft_figs/fig_2b_WTlegend.png', ppi = 500)

    #last 3 tracks only
    df = df.loc[df['AAsub'].isin(['Mis. Min.', 'Mis. Mean', 'Stop'])]
    all_pos = list(range(1,778))
    
    short_map = alt.Chart(df).mark_rect().encode(
    x = alt.X('AApos:N', axis=alt.Axis(
            values=[i for i in range(0, 777, 100)],
            title='Amino Acid Residue', titleFontSize = 32, labelFontSize = 28,
            labels=True,
            ticks= True,
            domain=True
    ),
              scale = alt.Scale(domain = all_pos
                               )
             ),
    y = alt.Y('AAsub:N', sort = order,
              axis = alt.Axis(title = '', titleFontSize = 32, labelFontSize = 28, ticks = False)),
    color = alt.condition(
        'datum.is_wt == true',
        alt.value('#000000'),  # Wild type in black
        alt.Color('score:Q', title = 'SGE Score',
                  scale = alt.Scale(
                      domain = [-0.3, 0, 0.15],  # Add upper bound for positive values
                      range = ['#ff0000', '#a6a6a6', '#a6a6a6']  # Keep gray for 0 and positive values
                  ),
                  legend = alt.Legend(titleFontSize = 24, 
                                      labelFontSize = 22)
        )
    ),
    tooltip =[alt.Tooltip('AApos', title = 'Position: '),
               alt.Tooltip('full_sub', title = 'Substitution: '),
               alt.Tooltip('score', title = 'SGE Score: ')]
    ).properties(
    height = 100, 
    width = 1750
    ).interactive()
    
    final_short_chart = alt.vconcat(annotations, short_map)
    final_short_chart = final_short_chart.configure_axis(
            grid = False
        ).configure_view(
            stroke = None
        ).resolve_axis(
            x = 'shared'
        )

    #final_short_chart.save('/Users/ivan/Desktop/BARD1_draft_figs/fig_2b_shortheatmap.png', ppi = 500)
    final_chart.display()
    final_short_chart.display()

In [ ]:
def final_fig(map, min_bar, mean_bar, stack):

    fig = (alt.vconcat(map, min_bar, mean_bar, spacing = 0).configure_view(strokeWidth = 0).configure_axis(domain = False))
    fig.display()

    fig_2 = alt.vconcat(map, stack, spacing = 0).resolve_scale(
        color = 'independent',
        y = 'independent'
    ).resolve_legend(
        color = 'independent'
    ).resolve_axis(
        x = 'independent', 
        y = 'independent'
    )
    fig_2.display()



In [ ]:
def main():
    raw_scores = read_scores(scores)
    sub_data = process_scores(raw_scores, func_cutoff, nf_cutoff)
    min_bar, min_df = get_min_res_score(raw_scores)
    mean_bar, mean_df = get_mean_res_score(raw_scores)
    min_mean_sub_data = append_mean_min(sub_data, min_df, mean_df)
    with_wt = add_wt_seq(min_mean_sub_data, BARD1_WT)
    
    #stacked_bar = min_mean_stacked(min_df, mean_df)
    #map = heatmap(sub_data)
    #heatmap(with_wt)
    
    heatmap_WT(with_wt)

    #min_mean_sub_data.to_excel('/Users/ivan/Desktop/test_excel_outputs/20250825_BARD1snvscores_heatmap.xlsx', index = False)
    #final_fig(map, min_bar, mean_bar, stacked_bar)

In [ ]:
main()